In [2]:
import gzip
import json

def messages_parse(data):
    data_with_no_binary = str(data).split("'")[1:-1][0]
    data_list =[i for i in data_with_no_binary.split('\\n') if i != '']
    messages_data = [json.loads(D) for D in data_list]
    return messages_data 

def messages_read(file="../data/coinbase_BTC-USD_20_10_06_000000-010000.json.gz"):
    file_object = gzip.open(file, "r")
    data = file_object.read()
    messages_data = messages_parse(data)
    return messages_data 

def snapshot_read(file="../data/coinbase_BTC-USD_20_10_06_00_00.json"):
    snaphsot_data = json.loads(open(file,'r').read())
    return snaphsot_data

def messages_filter(messages_data,initial_clob,final_clob):
    messages_data_filtered = [message_dict for message_dict in messages_data if message_dict['sequence'] >= initial_clob['sequence'] and message_dict['sequence'] <= final_clob['sequence']]
    return messages_data_filtered
            
messages_data = messages_read()
initial_clob = snapshot_read(file="../data/coinbase_BTC-USD_20_10_06_00_00.json")
final_clob = snapshot_read(file="../data/coinbase_BTC-USD_20_10_06_00_15.json")
messages_data_filtered = messages_filter(messages_data,initial_clob,final_clob)


In [3]:
import logging
import queue
#import data_load


logging.basicConfig(level=20)
logger = logging.getLogger()

class OrderDictTable:
    """
        Convert Dataset into Dictionary
    """
    def __init__(self, order_items):
        self.order_dict = {} 
        self.assign_orders_dict(order_items)
        
    def assign_orders_dict(self,order_items):
        keys = [subli[2] for subli in order_items]#map(lambda price,size,order_id: order_id,order_items)
        self.order_dict = dict([(order_id,order_item) for (order_id,order_item) in zip(keys,order_items) ])
    
    def get_values(self):
        return self.order_dict.values()

    def get_obj(self):
        return self.order_dict

    
class MessageOrder:
    def __init__(self,message_obj,asks_obj,bids_obj):
        self.type = message_obj['type']
        self.side = message_obj['side']
        self.price = message_obj['price']
        self.remaining_size = message_obj['remaining_size']
        self.order_id = message_obj['order_id']
        self.bids_obj = bids_obj
        self.asks_obj = asks_obj
        
    def remove_order(self):
        self.bids_obj.pop(order_id,None)
        self.asks_obj.pop(order_id,None)
    
    def add_order(self):
        [order_id] = [price,size,order_id]
        
                clob_bids_or_ask_dict[order_id] = [price,size,order_id]

    def update_from_message(self):
        self.bids_obj.pop(order_id,None)
        return self.bids_obj,self.asks_obj 
    
class CLOB:
    def __init__(self,clob):
        self.bids_obj = OrderDictTable(clob['bids']).get_obj()
        self.asks_obj = OrderDictTable(clob['asks']).get_obj()
        
    def order_remove(self,clob_bids_or_ask_dict,order_id):
        removed_order = clob_bids_or_ask_dict.pop(order_id,None)
        
    def order_add(self,clob_bids_or_ask_dict,price,size,order_id):
        clob_bids_or_ask_dict[order_id] = [price,size,order_id]
        
    def update_from_message(self,message_obj):
        message_type = message_obj['type']
        message_side = message_obj['side']
        
        if message_type == 'done':
            self.order_remove(message_obj['order_id'])
        elif message_type == 'open':
            self.order_add(message_obj['price'],message_obj['remaining_size'],message_obj['order_id'])
        
    def get_clob(self):
        return {
            'bids':self.bids_obj.values(),
            'asks':self.asks_obj.values()
        }


class CLOBSync:
    def clob_sync(self,initial_clob,messages_data_filtered):
        clob = CLOB(initial_clob)
        q = queue.Queue()
        messages_queue_data = sorted(messages_data_filtered,key=lambda i: i['sequence'])

        list(map(q.put,messages_queue_data ))
        while not q.empty():
            message_obj = q.get()
            clob.update_from_message(message_obj)
        final_clob_processed = clob.get_clob()
        return final_clob_processed

    
updated_clob = CLOBSync().clob_sync(initial_clob,messages_data_filtered)


In [20]:

def messages_from_order_id(messages_data,order_id):
    return [D for D in messages_data if 'order_id' in D.keys() and D['order_id'] == order_id]

def match_messages_from_order_id(messages_data,order_id):
    maker_orders = [D for D in messages_data if 'type' == 'match' and D['maker_order_id'] == order_id]
    taker_orders = [D for D in messages_data if 'type' == 'match' and D['taker_order_id'] == order_id]
    return maker_orders + taker_orders 

def orders_from_order_id(orders,order_id):
    return [subli for subli in orders if subli[2] == order_id]

def clob_length_test(final_clob,updated_clob,node='bids'):
    final_count = len(final_clob[node])
    updated_count = len(updated_clob[node])
    is_equal = final_count == updated_count
    return is_equal

def list_difference_find(l1,l2):
    return [order_subli for order_subli in l1 if order_subli not in l2]

def clob_difference_test(final_clob_orders,updated_clob_orders):
    difference_list_1 = list_difference_find(final_clob_orders,updated_clob_orders)
    difference_list_2 = list_difference_find(updated_clob_orders,final_clob_orders)
    return difference_list_1 + difference_list_2

def clob_difference_test_messages_debug_base(l1,l2,messages_data):
    difference_orders = list_difference_find(l1,l2)
    difference_orders_id = [order_subli[2] for order_subli in difference_orders]
    order_messages = [messages_from_order_id(messages_data,order_id) for order_id in difference_orders_id]
    order_messages_match = [messages_from_order_id(messages_data,order_id) for order_id in difference_orders_id]

    return order_messages + order_messages_match

def clob_difference_test_messages_debug(final_clob_orders,updated_clob_orders,messages_data):
    missing_order_messages = clob_difference_test_messages_debug_base(final_clob_orders,updated_clob_orders,messages_data)
    return missing_order_messages
    
def run_tests(final_clob,updated_clob):
    bids_difference = clob_difference_test(final_clob['bids'],updated_clob['bids'])
    asks_difference = clob_difference_test(final_clob['asks'],updated_clob['asks'])

    bids_length_equal = clob_length_test(final_clob,updated_clob,node='bids')
    asks_length_equal= clob_length_test(final_clob,updated_clob,node='asks')
    
    logger.info('bids_difference: %s', str(bids_difference))
    logger.info('asks_difference: %s', str(asks_difference))

    logger.info('bids_length_equal: %s', str(bids_length_equal))
    logger.info('asks_length_equal: %s', str(asks_length_equal))

    
missing_order_messages = clob_difference_test_messages_debug(final_clob['bids'],updated_clob['bids'],messages_data_filtered)

#run_tests(final_clob,updated_clob)

In [21]:
for order_message in missing_order_messages[0]:
    print (order_message)

{'type': 'open', 'side': 'buy', 'product_id': 'BTC-USD', 'time': '2020-10-06T00:14:28.127788Z', 'sequence': 16713045832, 'price': '10788.93', 'order_id': '4c53de3f-b8e4-4538-9efb-11d324615719', 'remaining_size': '3.1'}
{'type': 'received', 'side': 'buy', 'product_id': 'BTC-USD', 'time': '2020-10-06T00:14:28.127788Z', 'sequence': 16713045831, 'order_id': '4c53de3f-b8e4-4538-9efb-11d324615719', 'order_type': 'limit', 'size': '3.1', 'price': '10788.93', 'client_oid': '20201006-0000-0000-0002-000000003995'}
{'type': 'done', 'side': 'buy', 'product_id': 'BTC-USD', 'time': '2020-10-06T00:15:08.632549Z', 'sequence': 16713058257, 'order_id': '4c53de3f-b8e4-4538-9efb-11d324615719', 'reason': 'canceled', 'price': '10788.93', 'remaining_size': '2.80522372'}


In [15]:
print (len(messages_data_filtered))

252618
